In [1]:
import sys
sys.path.append('../')

# avg last freeze date
import pandas as pd
import noaa_datasets

In [2]:
# pull down master data
stations = noaa_datasets.Stations()

Success!


In [3]:
inventory = noaa_datasets.Inventory(stations)

Success!


In [4]:
# figure out best datasets to use for our requirement
# these are ones with the lastyear = 2019
# and at least 30 years for YearCount
# also, we're only looking at low temps, which is TMIN
df = inventory.df
df = df[ df['Element']=='TMIN' ]
df = df[ df['YearCount']>30 ]
df = df[ df['LastYear']==2019 ]

# whole world have 8,000+ results
# let's filter to just the united states, still 4,311...
df = df[ df['CountryCode']=='US' ]
df = df[ df['State']=='NE' ]

# let's pick a random 100 of these
df_samples = df.sample(3)

In [5]:
# now we need to pull the daily file information for 
# each of these 100 records
# this will take some time, which is why we reduced
# to just 100 in the first place!

# let's try the process with just a single station
# then we can iterate and concat
def c_to_f(row):
    # (0°C × 9/5) + 32 = 32°F
    return ( ((((row['VALUE']/10) * 9) / 5) + 32))

full_df = pd.DataFrame()

for station in df_samples['StationID']:
    df_daily = noaa_datasets.DailyFile(station)
    df_daily.df = df_daily.df[ df_daily.df['ELEMENT']=='TMIN' ]
    df_daily.df['Low Temp (f)'] = df_daily.df.apply(c_to_f, axis=1)
    df_daily.df = df_daily.df[ df_daily.df['YEAR']>2010 ]
    full_df = pd.concat([full_df, df_daily.df])

Retrieving USC00257070.dly
Transposing USC00257070 dataset
Applying data prep operations
Success!
Retrieving USC00250435.dly
Transposing USC00250435 dataset


C:\Users\Nate\Anaconda3\lib\site-packages\pandas\io\parsers.py:741: UserWarning: Duplicate names specified. This will raise an error in the future.
  return _read(filepath_or_buffer, kwds)


Applying data prep operations
Success!
Retrieving USC00253605.dly
Transposing USC00253605 dataset
Applying data prep operations
Success!


In [6]:
full_df = pd.merge(full_df, stations.df, left_on='StationID', right_index=True)

In [7]:
full_df.to_csv('NOAA Low Daily Temperatures Analysis.csv', sep='\t')